<a href="https://colab.research.google.com/github/AlinaRodina99/diplom/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import pandas as pd
import os
from skimage import io
from PIL import Image, ImageFile
import numpy as np
import torch
import torch.optim as optim

import matplotlib.pyplot as plt

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
new_image_transforms = transforms.Compose(
    [transforms.Resize(224), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])])

In [ ]:
t0 = transforms.Compose(
    [transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.4489, 0.3996, 0.3405],
                                     std=[0.2815, 0.2766, 0.2672])])
t1 = transforms.RandomPerspective(distortion_scale = 0.4, p = 1.0)
t2 = transforms.RandomHorizontalFlip(0.5)
t3 = transforms.RandomRotation((0, 180))

In [ ]:
image_transforms = [t1,t2, t3]

In [ ]:
import base64, csv

In [ ]:
directory = '/content/drive/MyDrive/images_init'

new_directory = '/content/drive/MyDrive/new_images'

list_dirs = os.listdir(directory)

for painter_folder in os.listdir(new_directory):
      for painting in os.listdir(new_directory + '/' + painter_folder):
        os.remove(new_directory + '/' + painter_folder + '/'  + painting)

In [ ]:
import io

In [ ]:
directory = '/content/drive/MyDrive/images_init'

new_directory = '/content/drive/MyDrive/new_images'

list_dirs = os.listdir(directory)

count = 0

for painter_folder in list_dirs[10:20]:
   for year_folder in os.listdir(directory + '/' + painter_folder):
      for painting in os.listdir(directory + '/' + painter_folder + '/' + year_folder):
        with open(f'{new_directory}/{painter_folder}/{painting}', 'wb') as new_file:
            image = Image.open(directory + '/' + painter_folder + '/' + year_folder + '/' + painting).convert("RGB")
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            byte_im = buf.getvalue()
            new_file.write(byte_im)
        with open("/content/drive/MyDrive/new_data_labels.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([f'{painter_folder}/{painting}', count])
        for i in range(len(image_transforms)):
          image = Image.open(directory + '/' + painter_folder + '/' + year_folder + '/' + painting).convert("RGB")
          image = image_transforms[i](image)
          with open(f'{new_directory}/{painter_folder}/{painting}_{i}', 'wb') as new_file:
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            byte_im = buf.getvalue()
            new_file.write(byte_im)
          with open("/content/drive/MyDrive/new_data_labels.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([f'{painter_folder}/{painting}_{i}', count])
   count += 1

In [ ]:
for painter_folder in list_dirs[60:72]:
   for year_folder in os.listdir(directory + '/' + painter_folder):
      for painting in os.listdir(directory + '/' + painter_folder + '/' + year_folder):
        with open(f'{new_directory}/{painter_folder}/{painting}', 'wb') as new_file:
            image = Image.open(directory + '/' + painter_folder + '/' + year_folder + '/' + painting).convert("RGB")
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            byte_im = buf.getvalue()
            new_file.write(byte_im)
        with open("/content/drive/MyDrive/new_data_labels.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([f'{painter_folder}/{painting}', count])
        for i in range(len(image_transforms)):
          image = Image.open(directory + '/' + painter_folder + '/' + year_folder + '/' + painting).convert("RGB")
          image = image_transforms[i](image)
          with open(f'{new_directory}/{painter_folder}/{painting}_{i}', 'wb') as new_file:
            buf = io.BytesIO()
            image.save(buf, format='JPEG')
            byte_im = buf.getvalue()
            new_file.write(byte_im)
          with open("/content/drive/MyDrive/new_data_labels.csv", mode="a", encoding='utf-8') as w_file:
            file_writer = csv.writer(w_file, delimiter = ",")
            file_writer.writerow([f'{painter_folder}/{painting}_{i}', count])
   count += 1

In [ ]:
torch.manual_seed(17)

In [ ]:
class PaintingsAndPainters(Dataset):
  def __init__(self, csv_file, root_dir, transform):
    self.annotations = pd.read_csv(csv_file, header=None)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    y_label = torch.tensor(self.annotations.iloc[index, 1])
    img_path = self.root_dir + self.annotations.iloc[index, 0]
    image = Image.open(img_path).convert("RGB")
    #transform = transforms.Grayscale()
    if self.transform:
      image = self.transform(image)
    return (image, y_label)



In [ ]:
dataset = PaintingsAndPainters(csv_file= "/content/drive/MyDrive/new_data_labels.csv", root_dir = '/content/drive/MyDrive/new_images/', transform = new_image_transforms)

In [ ]:
m = len(dataset)

In [ ]:
print(m)

20000


In [ ]:
from torch.utils.data import DataLoader,random_split
import torch

In [ ]:
generator_ = torch.Generator().manual_seed(17)
train_data, val_data = random_split(dataset, [int(m-m*0.2), int(m*0.2)], generator = generator_)

In [ ]:
batch_size=32

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True, num_workers = 1)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle = False, num_workers = 1)

In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
print(torch.cuda.device_count())

1


In [ ]:
lr = 0.001

In [ ]:
from torch import nn

In [ ]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Linear(2048, 72).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 78.9MB/s]


In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.fc.parameters(), lr=lr, momentum = 0.9)

In [ ]:
#model = nn.DataParallel(model)

In [ ]:
val_loss_prev = 0

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/new_model.pt", map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])
val_loss_prev = checkpoint['loss']
counter = checkpoint['counter']
lr = checkpoint['lr']
epoch = checkpoint['epoch']
val_loss_prev
epoch

23

In [ ]:
from torch import autocast

In [ ]:
def train_epoch(autoencoder, device, dataloader, loss_fn, optimizer, epoch):
    # Set train mode for both the encoder and the decoder
    train_loss = []
    model.train()
    avg_train_loss = 0
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch, labels in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        image_batch = image_batch.to(device)
        labels = labels.to(device)
        # Evaluate loss
        outputs = model(image_batch)
        loss = loss_fn(outputs, labels)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print batch loss
        print('\t partial train loss (single batch): %f' % (loss.data))
        avg_train_loss += loss / len(dataloader)
    return avg_train_loss

In [ ]:
device

device(type='cuda')

In [ ]:
val_losses = []
val_accs = []
def test_epoch(autoencoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    model.eval()
    avg_val_loss = 0
    predicted_labels = []
    real_labels = []
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, labels in dataloader:
            # Move tensor to the proper device
            #if torch.cuda.is_available():
               #print('yes')
            image_batch = image_batch.to(device)
            labels = labels.to(device)
            # Decode data
            outputs = model(image_batch)
            # Append the network output and the original image to the lists
            loss = loss_fn(outputs, labels)
            avg_val_loss += loss / len(dataloader)
            predicted_labels.append(outputs.argmax(dim = 1))
            real_labels.append(labels)
    predicted_labels = torch.cat(predicted_labels)
    real_labels = torch.cat(real_labels)
    val_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()
    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)
    with open ('/content/drive/MyDrive/file.txt', 'a') as file:
      file.write(str(avg_val_loss) + ' ' + str(val_acc) + '\n')
    return avg_val_loss, val_acc

In [ ]:
num_epochs = 30
counter = 0
for epoch in range(23, num_epochs):
   train_loss =train_epoch(model,device, train_loader,loss_fn, optim, epoch)
   val_loss_acc = test_epoch(model, device,valid_loader,loss_fn)
   val_loss = val_loss_acc[0]
   val_acc = val_loss_acc[1]
   
   torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': val_loss,
            'counter': counter,
            'lr' : lr,
            'epoch': epoch,
            }, "/content/drive/MyDrive/new_model.pt")
   if epoch > 0:
    
     checkpoint = torch.load("/content/drive/MyDrive/new_model.pt")
     val_loss_prev = checkpoint['loss']
     if val_loss_prev - val_loss <= 0:
       counter += 1
       if counter > 7:
         break
   print('\n EPOCH {}/{} \t train loss {} \t val loss {} \t val acc {}'.format(epoch + 1, num_epochs,train_loss,val_loss, val_acc))

	 partial train loss (single batch): 0.894231
	 partial train loss (single batch): 1.110598
	 partial train loss (single batch): 1.566516
	 partial train loss (single batch): 1.103347
	 partial train loss (single batch): 0.918445
	 partial train loss (single batch): 0.844904
	 partial train loss (single batch): 1.208780
	 partial train loss (single batch): 0.846893
	 partial train loss (single batch): 1.449888
	 partial train loss (single batch): 1.073278
	 partial train loss (single batch): 1.041200
	 partial train loss (single batch): 1.253083
	 partial train loss (single batch): 1.163072
	 partial train loss (single batch): 1.349046
	 partial train loss (single batch): 0.898289
	 partial train loss (single batch): 0.765521
	 partial train loss (single batch): 0.929030
	 partial train loss (single batch): 0.908542
	 partial train loss (single batch): 1.091279
	 partial train loss (single batch): 1.123536
	 partial train loss (single batch): 1.121943
	 partial train loss (single batch

In [ ]:
predicted_labels = []
real_labels = []
model.eval()
with torch.no_grad():
  for image_batch, labels in train_loader:
    image_batch = image_batch.to(device)
    labels = labels.to(device)
    y_predicted = model(image_batch)
    predicted_labels.append(y_predicted.argmax(dim = 1))
    real_labels.append(labels)
predicted_labels = torch.cat(predicted_labels)
real_labels = torch.cat(real_labels)
train_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

In [ ]:
train_acc

tensor(0.5287)

In [ ]:
predicted_labels = []
real_labels = []
model.eval()
with torch.no_grad():
  for image_batch, labels in valid_loader:
    image_batch = image_batch.to(device)
    labels = labels.to(device)
    y_predicted = model(image_batch)
    predicted_labels.append(y_predicted.argmax(dim = 1))
    real_labels.append(labels)
predicted_labels = torch.cat(predicted_labels)
real_labels = torch.cat(real_labels)
val_acc = (predicted_labels == real_labels).type(torch.FloatTensor).mean()

/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


In [ ]:
val_acc

tensor(0.4805)